# Simple ETL Application

In this notebook, we connect to an api, extract data, create a spark dataframe and then finally write to a postgres database. 

In [ ]:
import requests
import json
import pandas as pd
import psycopg2

## Get data from Randomuser API

In [2]:
def get_data():
    """
    get data from random user api
    """
    try:
        url = "https://randomuser.me/api/"
        response = requests.get(url)
        response = json.loads(response.text)
        return response["results"][0]
    except Exception as e:
        print(f"there was an error{e}")
        return None

## Create dataframe

In [3]:
people_list = []

for _ in range(5):
    people_list.append(get_data())

data = pd.DataFrame(people_list,index=None)

## Create table in postgres Database using psycopg2 library

In [ ]:
query = """CREATE TABLE test_table (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    email TEXT UNIQUE
)"""

In [ ]:
# url
url = "http://postgres-db:45432/postgres_db"
#user
username = "postgres"
password = "postgres"

conn_details = psycopg2.connect(
    host='postgres-db',
    database="postgres_db",
    user=username,
    # secretlint-disable
    password=password,
    # secretlint-enable
    port='5432'
)

with conn_details:
    cursor = conn_details.cursor()
    query = query
    cursor.execute(query)

## Insert Records

## Create a spark Session

In [24]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YourAppName") \
    .config("spark.driver.extraClassPath", "postgresql-connector-java.jar") \
    .config("spark.executor.extraClassPath", "postgresql-connector-java.jar") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/09 21:05:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Create Spark dataframe

In [25]:
spark_df = spark.createDataFrame(data)